In [1]:
import torch
import torch.nn as nn
from typing import Type, Any, Callable, Union, List, Dict, Optional, cast
from torch import Tensor
from collections import OrderedDict 

In [2]:
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url

In [3]:
import torch.nn as nn
import torch.optim as optim

In [4]:
from torchvision.models.resnet import *
from torchvision.models.resnet import BasicBlock, Bottleneck

In [5]:
from torchvision.models.resnet import ResNet50_Weights

In [6]:
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import torchvision.transforms as T
from torchvision import datasets, models, transforms

In [7]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Daten laden

In [8]:
train_dir = "./dataset/train/"
test_dir = "./dataset/test/"
train_classa_dir = "./dataset/train/moire/"
train_classb_dir = "./dataset/train/no_moire/"
test_classa_dir = "./dataset/test/moire/"
test_classb_dir = "./dataset/test/no_moire/"

In [9]:
# Create transform function
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(), # data augmentation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalization
])
transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [10]:
train_dataset = datasets.ImageFolder(train_dir, transforms_train)
test_dataset = datasets.ImageFolder(test_dir, transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=12, shuffle=True, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=12, shuffle=False, num_workers=0)

# Modell

In [11]:
class IntResNet(ResNet):
    def __init__(self,output_layer,*args):
        self.output_layer = output_layer
        super().__init__(*args)
        
        self._layers = []
        for l in list(self._modules.keys()):
            self._layers.append(l)
            if l == output_layer:
                break
        self.layers = OrderedDict(zip(self._layers,[getattr(self,l) for l in self._layers]))

    def _forward_impl(self, x):
        for l in self._layers:
            x = self.layers[l](x)

        return x

    def forward(self, x):
        return self._forward_impl(x)

In [12]:
def new_resnet(
    arch: str,
    outlayer: str,
    block: Type[Union[BasicBlock, Bottleneck]],
    layers: List[int],
    pretrained: bool,
    progress: bool,
    **kwargs: Any
):
    '''model_urls = {
        'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
        'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
        'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
        'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
        'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
        'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
        'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
        'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
        'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
    }'''
    model = IntResNet(outlayer, block, layers, **kwargs)
    
    if pretrained:
        state_dict = load_state_dict_from_url(ResNet50_Weights.IMAGENET1K_V2.url,progress=progress)
        model.load_state_dict(state_dict)
    
    return model

In [13]:
class TinyModel(torch.nn.Module):

    def __init__(self):
        super(TinyModel, self).__init__()
        self.resnet = new_resnet('resnet50','layer2',Bottleneck, [3, 4, 6, 3],True,True)
        self.flatten = torch.nn.Flatten()
        self.linear1 = torch.nn.Linear(401408, 500)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(500, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.resnet(x)
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.sigmoid(x)
        return x

tinymodel = TinyModel()

In [14]:
model = TinyModel()

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

# Train

In [16]:
# Define loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

In [17]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
import time

In [19]:
# res net freezen
for param in list(model.children())[0].parameters():
    param.requires_grad = False

In [20]:
#### Train model
train_loss=[]
train_accuary=[]
test_loss=[]
test_accuary=[]

num_epochs = 30   #(set no of epochs)
start_time = time.time() #(for showing time)
# Start loop
for epoch in range(num_epochs): #(loop for every epoch)
    print("Epoch {} running".format(epoch)) #(printing message)
    """ Training Phase """
    model.train()    #(training model)
    running_loss = 0.   #(set loss 0)
    running_corrects = 0 
    # load a batch data of images
    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.reshape((-1,1)).float().to(device)
        # forward inputs and get output
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        # get loss value and update the network weights
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        running_corrects += torch.sum(labels.data == outputs.round()).item()
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset) * 100.
    # Append result
    train_loss.append(epoch_loss)
    train_accuary.append(epoch_acc)
    # Print progress
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch+1, epoch_loss, epoch_acc, time.time() -start_time))
    """ Testing Phase """
    model.eval()
    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.reshape((-1,1)).float().to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            running_corrects += torch.sum(labels.data == outputs.round()).item()
        epoch_loss = running_loss / len(test_dataset)
        epoch_acc = running_corrects / len(test_dataset) * 100.
        # Append result
        test_loss.append(epoch_loss)
        test_accuary.append(epoch_acc)
        # Print progress
        print('[Test #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch+1, epoch_loss, epoch_acc, time.time()- start_time))

Epoch 0 running
[Train #1] Loss: 0.0661 Acc: 58.5938% Time: 14.1584s
[Test #1] Loss: 0.0865 Acc: 46.9136% Time: 15.6410s
Epoch 1 running
[Train #2] Loss: 0.0475 Acc: 74.6094% Time: 25.7054s
[Test #2] Loss: 0.0281 Acc: 90.1235% Time: 27.3563s
Epoch 2 running
[Train #3] Loss: 0.0383 Acc: 79.6875% Time: 37.1541s
[Test #3] Loss: 0.0405 Acc: 91.3580% Time: 38.9033s
Epoch 3 running
[Train #4] Loss: 0.0349 Acc: 82.0312% Time: 49.0883s
[Test #4] Loss: 0.0489 Acc: 64.1975% Time: 50.7117s
Epoch 4 running
[Train #5] Loss: 0.0358 Acc: 81.2500% Time: 60.8906s
[Test #5] Loss: 0.0558 Acc: 93.8272% Time: 62.5478s
Epoch 5 running
[Train #6] Loss: 0.0390 Acc: 80.4688% Time: 73.1314s
[Test #6] Loss: 0.0204 Acc: 92.5926% Time: 74.5074s
Epoch 6 running
[Train #7] Loss: 0.0419 Acc: 78.1250% Time: 83.5300s
[Test #7] Loss: 0.0263 Acc: 91.3580% Time: 85.0217s
Epoch 7 running
[Train #8] Loss: 0.0423 Acc: 79.6875% Time: 94.2561s
[Test #8] Loss: 0.0208 Acc: 91.3580% Time: 95.8915s
Epoch 8 running
[Train #9] Loss:

KeyboardInterrupt: 

In [ ]:
import plotly.express as px
import numpy as np

In [ ]:
model.resnet.layers['layer1'][1].conv1.weight.shape

In [ ]:
fig = px.line(
    x=np.arange(1,num_epochs+1),
    y=train_accuary
)

# Only thing I figured is - I could do this 
fig.add_scatter(
    x=np.arange(1,num_epochs+1),
    y=test_accuary
) # Not what is desired - need a line 

# Show plot 
fig.show()